<a href="https://colab.research.google.com/github/juooo1117/cyber_security_project/blob/main/cyber_security_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **데이터 로드**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score

# **데이터 전처리**

In [4]:
df = pd.read_csv('/content/drive/MyDrive/A_Track_trainset.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Log_Number    45000 non-null  int64 
 1   payload       45000 non-null  object
 2   label_action  45000 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [6]:
df.head(20)

,Log_Number,payload,label_action
0,0,"GET /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,POST /owa/auth/logon.aspx?replaceCurrent=1%22%...,System_Cmd_Execution
2,2,GET /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,3,GET / HTTP/1.1\r\n\r\n,HOST_Scan
4,4,GET /sub_04_1_read.php?page=1&id=31%29%3BSELEC...,System_Cmd_Execution
5,5,POST /member/login HTTP/1.1\r\nContent-Length:...,Vulnerability_Scan
6,6,GET /board/?category=&gdviewer=1&goods_seq=69&...,System_Cmd_Execution
7,7,GET /board/board_list?code=welfare&search_type...,SQL_Injection
8,8,GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,HOST_Scan
9,9,GET / HTTP/1.1\r\n\r\n,HOST_Scan


In [7]:
df.columns

Index(['Log_Number', 'payload', 'label_action'], dtype='object')

In [8]:
# label 종류('label_action') -> 총 9개
df['label_action'].unique()

array(['System_Cmd_Execution', 'SQL_Injection', 'HOST_Scan',
       'Vulnerability_Scan', 'Path_Disclosure', 'Cross_Site_Scripting',
       'Leakage_Through_NW', 'Directory_Indexing',
       'Automatically_Searching_Infor'], dtype=object)

In [11]:
# label category의 각각 값 개수
df['label_action'].value_counts()

Vulnerability_Scan               16867
System_Cmd_Execution              9807
HOST_Scan                         6315
Path_Disclosure                   4775
SQL_Injection                     3395
Cross_Site_Scripting              1348
Automatically_Searching_Infor     1119
Leakage_Through_NW                 956
Directory_Indexing                 418
Name: label_action, dtype: int64

In [10]:
# Log_Number column 삭제
df = df.drop('Log_Number', axis = 1)
df.head(20)

,payload,label_action
0,"GET /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,POST /owa/auth/logon.aspx?replaceCurrent=1%22%...,System_Cmd_Execution
2,GET /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,GET / HTTP/1.1\r\n\r\n,HOST_Scan
4,GET /sub_04_1_read.php?page=1&id=31%29%3BSELEC...,System_Cmd_Execution
5,POST /member/login HTTP/1.1\r\nContent-Length:...,Vulnerability_Scan
6,GET /board/?category=&gdviewer=1&goods_seq=69&...,System_Cmd_Execution
7,GET /board/board_list?code=welfare&search_type...,SQL_Injection
8,GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,HOST_Scan
9,GET / HTTP/1.1\r\n\r\n,HOST_Scan


In [12]:
# 중복값제거
# payload 가 같으나 label_action 이 다른 데이터 -> 해당 payload의 label_action별 카운트 중 빈도가 높은 label_action으로 변경
#                                                                        카운트가 같다면 전체 label_action 별 카운트 빈도가 높은 label_action으로 변경

drop_df = df.drop_duplicates()
x = drop_df[drop_df.duplicated(['payload'], keep=False)].drop_duplicates(subset = ['payload'])['payload']

for i in x:

    v = df[df['payload'] == i]['label_action'].mode().values

    if len(v) == 1:
        df.loc[(df['payload'] == i), 'label_action'] = v[0]

    else:
        fvc = 0
        for j in df[df['payload'] == i]['label_action'].unique():
            vc = df['label_action'].value_counts().loc[j]
            if vc > fvc :
                fvc = vc
                m = j
            else :
                pass
        df.loc[(df['payload'] == i), 'label_action'] = m

In [13]:
# 각 label_action 값 변화된 것 확인 -> 총 개수는 동일(45000개)
df['label_action'].value_counts()

Vulnerability_Scan               16882
System_Cmd_Execution              9797
HOST_Scan                         6315
Path_Disclosure                   4771
SQL_Injection                     3394
Cross_Site_Scripting              1348
Automatically_Searching_Infor     1119
Leakage_Through_NW                 956
Directory_Indexing                 418
Name: label_action, dtype: int64

In [ ]:
# label_action: Vulnerability_Scan 인 값들 (취약한 값들 & 제일 값 많음 -> 맨 나중에??)
# 나머지 label_action category 별로도 payload 값을 확인했으나, 기준으로 삼을만한 패턴이 보이지 않음

df.loc[df['label_action'] == 'Vulnerability_Scan', ['label_action','payload']]

In [26]:
# 한글과 숫자를 마스킹처리
df['payload'] = df['payload'].str.replace(r'[ㄱ-ㅣ가-힣]+', '<KOREAN>').str.replace(r'[0-9]+', '<NUM>')

# 특수문자 제거 -> 공백으로 변환 & 소문자로 변환
df['payload'] = df['payload'].str.replace(r'[^a-zA-Z<>]', ' ')
df['payload'] = df['payload'].str.lower()

<ipython-input-26-e91b6a1e9449>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['payload'] = df['payload'].str.replace(r'[ㄱ-ㅣ가-힣]+', '<KOREAN>').str.replace(r'[0-9]+', '<NUM>')
<ipython-input-26-e91b6a1e9449>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['payload'] = df['payload'].str.replace(r'[^a-zA-Z<>]', ' ')


In [28]:
df.head(5)

,payload,label_action
0,get forum<num> professionnel asp n ...,System_Cmd_Execution
1,post owa auth logon aspx replacecurrent <num>...,System_Cmd_Execution
2,get goods goods search display type list arr ...,SQL_Injection
3,get http <num> <num> r n r n,HOST_Scan
4,get sub <num> <num> read php page <num> id <n...,System_Cmd_Execution


In [29]:
# df['payload'] -> df_text로 list
# df['label_action'] -> df_label로 array

df_text = list(df['payload'])
df_label = np.array(df['label_action'])

In [30]:
# CountVectorizer로 벡터화

from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(analyzer="word", max_features=10000)

train_features=vectorizer.fit_transform(df_text)

In [31]:
train_features.shape

(45000, 10000)

# **모델링 & 정확도확인**

In [32]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [33]:
# 훈련 데이터 셋과 검증 데이터 셋으로 분리

TEST_SIZE = 0.3
RANDOM_SEED = 1117

train_x, val_x, train_y, val_y = train_test_split(train_features, df_label, test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [34]:
# RandomForestClassifier (DecisionTreeClassifier는 정확도가 낮아서 사용X)
# n_estimators : boosting을 반복할 횟수(손실함수의 가중치를 몇 번이나 조정할 것인지)
#                값이 클수록 정확도가 증가하나, overfitting의 가능성 또한 증가

forest = RandomForestClassifier(n_estimators=100, random_state = RANDOM_SEED, class_weight = 'balanced')
forest.fit(train_x, train_y)

RandomForestClassifier(class_weight='balanced', random_state=1117)

In [35]:
# val data로 모델 accuracy 검증
forest.score(val_x, val_y)

0.8440740740740741

In [42]:
# .score(val_x, val_y) 라이브러리를 풀어쓴 코드 (동일한 accuracy 결과 출력!)
from sklearn.metrics import accuracy_score

predicted = forest.predict(val_x)
accuracy = accuracy_score(val_y, predicted)
print(accuracy)

0.8440740740740741


In [36]:
# crosstab: 각 'label_action' category별로 'pred' & 'real' 값을 교차 확인해서 피팅한 모델로 예측한 값이 얼마나 맞았는지 확인 (diagonal은 맞춘것!)
pred = forest.predict(val_x)
crosstab = pd.crosstab(val_y, pred, rownames=['real'], colnames=['pred'])
crosstab

pred,Automatically_Searching_Infor,Cross_Site_Scripting,Directory_Indexing,HOST_Scan,Leakage_Through_NW,Path_Disclosure,SQL_Injection,System_Cmd_Execution,Vulnerability_Scan
real,,,,,,,,,
Automatically_Searching_Infor,360,0,0,0,0,0,0,0,1
Cross_Site_Scripting,0,329,0,4,0,4,1,41,6
Directory_Indexing,0,0,122,0,0,0,0,4,4
HOST_Scan,1,9,2,1576,0,30,0,81,132
Leakage_Through_NW,1,0,1,1,301,0,0,0,0
Path_Disclosure,3,3,1,21,0,1253,0,65,88
SQL_Injection,0,1,2,0,0,1,841,112,35
System_Cmd_Execution,0,55,5,60,0,90,127,2286,340
Vulnerability_Scan,4,9,2,213,0,94,49,402,4327


In [37]:
# 안전성 높이기 위해서 새로운 위험요소에 대한 가정을 추가하자
preds = forest.predict(val_x)
probas = forest.predict_proba(val_x)
print(preds.shape)
print(probas.shape)

(13500,)
(13500, 9)


In [38]:
# 예측치의 예측 확률이 0.7 이하인 경우(확신이 없을 경우) outlier로 판단
preds[np.where(np.max(probas, axis=1) < 0.7)] = 7
new_crosstab = pd.crosstab(val_y, preds, rownames=['real'], colnames=['pred'])
new_crosstab

pred,7,Automatically_Searching_Infor,Cross_Site_Scripting,Directory_Indexing,HOST_Scan,Leakage_Through_NW,Path_Disclosure,SQL_Injection,System_Cmd_Execution,Vulnerability_Scan
real,,,,,,,,,,
Automatically_Searching_Infor,2,359,0,0,0,0,0,0,0,0
Cross_Site_Scripting,65,0,295,0,2,0,1,0,22,0
Directory_Indexing,6,0,0,118,0,0,0,0,3,3
HOST_Scan,291,0,4,1,1462,0,13,0,23,37
Leakage_Through_NW,2,1,0,0,1,300,0,0,0,0
Path_Disclosure,271,0,1,0,3,0,1093,0,33,33
SQL_Injection,174,0,0,0,0,0,0,752,53,13
System_Cmd_Execution,701,0,23,0,15,0,32,58,1948,186
Vulnerability_Scan,763,2,3,0,117,0,21,23,190,3981


# **test data 답 예측**

In [ ]:
# test['full_log'] => test_text로 list

test_text=list(test['full_log'])
ids=list(test['id'])

In [ ]:
# test 데이터 벡터화

test_features = vectorizer.transform(test_text)

In [ ]:
results = forest.predict(test_features)
results_proba = forest.predict_proba(test_features)

In [ ]:
# level이 2,4,6이 아니고(해당 category 값이 너무 적어서 outlier판단에서 제외) 예측치의 예측 확률이 0.7 이하, 즉 확신이 없을 경우 이상치로 판단
results_new = results.copy()
results_new[np.where((np.max(results_proba, axis=1) < 0.7) & (results != 6) & (results != 4) & (results != 2))] = 7

In [ ]:
# submission 저장
submission['level']=results_new

In [ ]:
submission['level'].value_counts()

In [ ]:
submission.to_csv('submission_csv/final.csv', index=False)